# Анализ предпочтений клиентов книжного сервиса

**Цель исследования:**

Проанализировать базу данных книжного сервиса для того, чтобы сформулировать ценностное предложение для нового продукта.

**Задачи исследования:**

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг среди книг с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Описание данных:**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.


## Настройка необходимых параметров

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# создадим функцию ддя вывода результатов sql-запроса
def result(query):
    con=engine.connect()
    return pd.io.sql.read_sql(sql=text(query), con = con)

## Знакомство с данными

Для начала выведем на экран первые строки всех таблиц базы данных:

In [3]:
query = \
'''
SELECT * 
FROM books 
LIMIT 5
'''
result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = \
'''
SELECT * 
FROM authors 
LIMIT 5
'''
result(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = \
'''
SELECT * 
FROM ratings
LIMIT 5
'''
result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = \
'''
SELECT * 
FROM reviews
LIMIT 5
'''
result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [8]:
query = \
'''
SELECT * 
FROM publishers
LIMIT 5
'''
result(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблицы выгружаются нормально, можно приступить к ответам на вопросы.

## Решение заданий

### Cколько книг вышло после 1 января 2000 года

In [9]:
query = \
'''
SELECT COUNT(book_id) 
FROM books
WHERE publication_date > '2000-01-01'
'''
result(query)

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [10]:
query = \
'''
SELECT b.book_id,
        b.title,
        COUNT(DISTINCT r.review_id) AS quantity_review,
        AVG(rt.rating) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS r ON b.book_id = r.book_id
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY b.book_id
ORDER BY quantity_review DESC, avg_rating DESC

'''
result(query)

,book_id,title,quantity_review,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,387,Leonardo's Notebooks,0,4.000000
997,221,Essential Tales and Poems,0,4.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


In [11]:
# проверка соответствия суммы ревью количеству всех ревью в бд
query = \
'''
SELECT SUM(kmk.quantity_review)
from (SELECT b.book_id,
        b.title,
        COUNT(distinct r.review_id) AS quantity_review,
        AVG(rt.rating) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS r ON b.book_id = r.book_id
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY b.book_id
ORDER BY quantity_review DESC, avg_rating DESC) as kmk

'''
result(query)

,sum
0,2793.0


Больше всего обзоров было сделано на книгу Twilight (Twilight #1) - всего 7 обзоров. Средний рейтинг этой книги - 3.66.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [12]:
query = \
'''
SELECT p.publisher,
    COUNT(b.book_id) AS book_count
FROM books AS b
LEFT JOIN publishers AS p on b.publisher_id = p.publisher_id
WHERE b.num_pages >50
GROUP BY p.publisher
ORDER BY book_count DESC
LIMIT 1
'''
result(query)

,publisher,book_count
0,Penguin Books,42


Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books, всего выпустило 42 книги.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [13]:
query = \
'''
SELECT tbl.author,
    AVG(tbl.avg_rate) AS avg_rate
FROM (SELECT a.author,
            b.book_id,
            COUNT(r.rating) AS rating_count,
            AVG(r.rating) AS avg_rate
        FROM authors AS a
        JOIN books AS b ON a.author_id = b.author_id
        JOIN ratings AS r ON b.book_id = r.book_id
        GROUP BY a.author, b.book_id
        HAVING COUNT(r.rating) >50) AS tbl
GROUP BY author
ORDER BY avg_rate DESC
LIMIT 1

'''
result(query)

,author,avg_rate
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг среди книг с 50 и более оценками - J.K. Rowling/Mary GrandPré, средний рейтинг - 4.28.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [14]:
query = \
'''
SELECT AVG(tbl.cnt_review)
FROM (SELECT username,
        COUNT(review_id) AS cnt_review
    FROM reviews
    WHERE username IN (SELECT username
                        FROM ratings
                        GROUP BY username
                        HAVING COUNT(rating) > 48)
    GROUP BY username) AS tbl

'''
result(query)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 обзора.

## Выводы

1) После 1 января 2000 года вышло 819 книг.
2) Больше всего обзоров было сделано на книгу Twilight (Twilight #1) - всего 7 обзоров. Средний рейтинг этой книги - 3.66.
3) Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books, всего выпустило 42 книги.
4) Автор с самой высокой средней оценкой книг среди книг с 50 и более оценками - J.K. Rowling/Mary GrandPré, средний рейтинг - 4.28.
5) Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 обзора.